# Run grid search

In [79]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Imbalance Sampling
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN

# Pipelines imports
import xgboost
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.feature_selection import GenericUnivariateSelect,f_regression
from sklearn.ensemble import IsolationForest
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split


# Scoring function
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import make_scorer
from sklearn.base import TransformerMixin,BaseEstimator
#from scipy import signal

# Keras imports
import tensorflow as tf
from tensorflow.keras import utils
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import sequence 
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Input, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv1D, BatchNormalization, GlobalAveragePooling1D, Bidirectional, MaxPooling1D

# Grid Search
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [184]:
# Get data
Xdf = pd.read_csv("X_train.csv")
XTestdf = pd.read_csv("X_test.csv")
ydf = pd.read_csv("y_train.csv")
X = Xdf[Xdf.columns[1:]].values
Xtest = XTestdf[XTestdf.columns[1:]].values
y = ydf[ydf.columns[1]].values
print(X.shape)
print(y.shape)

# Optional scaling
scaling=True
if scaling:
    from sklearn import preprocessing
    scaler = preprocessing.RobustScaler().fit(X)
    X = scaler.transform(X)
    Xtest = scaler.transform(Xtest)

# Use these for final training in case of test_train_split
X_all = np.copy(X)
y_all = np.copy(y)

# Split dataset before oversampling to prevent sampling from the 
split_data = True
if split_data:
    X, x_val, y, y_val = train_test_split(
        X, y, test_size=0.2)

# Accounting for imbalance in the data
oversample = False
undersample = False # Switch both to false to do nothing
if oversample:
    # Random Oversampler
    #ros = RandomOverSampler(random_state=42)
    #X_resampled, y_resampled = ros.fit_resample(X, y)
    
    # Smote oversampler
    X, y = SMOTE().fit_resample(X, y)
    
    # Adasyn oversampler
    #X_resampled, y_resampled = ADASYN().fit_resample(X, y)
elif undersample:
    # Count number of occurances per class
    _, counts = np.unique(y,return_counts=True)
    print(counts)

    # Divide by class 
    X0 = X[y != 1]
    X1 = X[y == 1]
    y0 = y[y != 1]
    y1 = y[y == 1]
    print(X0.shape)
    print(X1.shape)
    print(y0.shape)
    print(y1.shape)

    # Downsampling
    indices_subsampled = np.random.RandomState(seed=42).choice(range(X1.shape[0]),int(counts[0]),replace=False)
    X1_subsampled = X1[indices_subsampled]
    y1_subsampled = y1[indices_subsampled]
    print(X1_subsampled.shape)
    print(y1_subsampled.shape)

    X = np.concatenate((X0,X1_subsampled))
    y = np.concatenate((y0,y1_subsampled))

def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.RandomState(seed=42).permutation(len(a))
    return a[p], b[p]
X,y = unison_shuffled_copies(X,y)

one_hot = False
if one_hot:
    y = np_utils.to_categorical(y)
    
print(X.shape)
print(y.shape)



(4800, 1000)
(4800,)
(3840, 1000)
(3840,)


In [ ]:
# Function to get best estimator
def get_best_estimator(pipeline, X, y, parameters, scoring, cv=5, verbose=0, n_jobs=None):
    print('Finding best parameters through grid search...')
    grid = GridSearchCV(pipeline, param_grid=parameters, scoring=scoring, cv=cv, verbose=verbose, refit=True, return_train_score=False, n_jobs=n_jobs)
    grid.fit(X, y)
    print('Done!')
    return grid.best_estimator_, grid

def find_best(pipeline, X, y, parameters, scoring, cv=5, verbose=0, n_jobs=None):
    best_pipeline, grid = get_best_estimator(pipeline, X, y, parameters, scoring, cv=cv, verbose=verbose, n_jobs=n_jobs)
    return grid

# If you want to revert 1-hot encoding after classifier in the sklearn pipeline. NOT SURE IF IT WORKS
class Revert1Hot(BaseEstimator,TransformerMixin):

    # here you define the operation it should perform
    def transform(self, X, y=None, **fit_params):
        return np.argmax(X)

    # just return self
    def fit(self, X, y=None, **fit_params):
        return self

## OneVsRestClassifier

In [ ]:
score = make_scorer(balanced_accuracy_score)

# Different classifiers that need to be tested
#classifiers = [
#    KNeighborsClassifier(3),
#    SVC(kernel="rbf", C=0.025, probability=True),
#    NuSVC(probability=True),
#    DecisionTreeClassifier(),
#    RandomForestClassifier(),
#    AdaBoostClassifier(),
#    GradientBoostingClassifier()
#    ]

steps = [
    
    ('scaler', StandardScaler()), 
    ('ufs',GenericUnivariateSelect(score_func=f_regression, mode='k_best', param=200)),
    ('XGB', OneVsRestClassifier(xgboost.XGBClassifier(colsample_bytree=0.6,
                                                      min_child_weight=6,
                                                      max_depth=8)))
     ]

pipeline = Pipeline(steps)

parameters = {'ufs__param':[1000]}


grid = find_best(pipeline, X, y, parameters, score, cv=3, verbose=1, n_jobs=-1)

In [ ]:
pd.DataFrame(grid.cv_results_)[['mean_fit_time','mean_score_time','params','mean_test_score','std_test_score']]

In [ ]:
# Still not sure how to extract best model from GridSearch and predict it for the test set
pipeline.fit(X,y,scoring=score)
result = pipeline.predict(Xtest)

## Deep Learning Model

In [ ]:
# Not very promising as it only reaches 66.83% accuracy. 
# Maybe try to use convolutional network although doubting that could help.
def create_model():
    num_features=1000
    mid_size=100
    dropout=0.1
    
    # Model Definition
    model = Sequential()
    model.add(Dense(num_features, activation="relu", input_shape=(num_features,)))
    # model.add(Conv1D(filters=64, kernel_size=5, activation='relu', input_shape=(1000,)))
    model.add(Dropout(dropout))
    model.add(Dense(200, activation="relu"))
    model.add(Dropout(dropout))
    model.add(Dense(50, activation="relu"))
    model.add(Dropout(dropout))
    model.add(Dense(3, activation="softmax"))

    #compile model using accuracy to measure model performance
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'], 
                  weighted_metrics=['accuracy'])

    model.summary()
    return model

#Define Early stopping mechanism
es = EarlyStopping(monitor='val_loss', mode='max', patience=5, verbose=1)

# wrap the model using the function you created   , callbacks=[es]
clf = KerasClassifier(build_fn=create_model, epochs=20, batch_size=5, verbose=1)

In [ ]:
kfold = KFold(n_splits=3, shuffle=True)
results = cross_val_score(clf, X, y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [ ]:
score = make_scorer(balanced_accuracy_score)

# Having difficulties adding the tensorflow deep learning model to the sklearn pipeline. 
# It complains about the keras classifier not having a transform function although it is used in this way in tutorials. 
# Maybe I have a mistake somewhere.
steps = [
    
    ('scaler', StandardScaler()), 
    ('ufs',GenericUnivariateSelect(score_func=f_regression, mode='k_best', param=200)),
    ('clf', clf),
    ('revert1hot', Revert1Hot())
     ]

pipeline = Pipeline(steps)

#parameters = [{'ufs__param':[200], 'clf__features':[200], 'clf__mid_size':[50, 10]},
#             {'ufs__param':[500], 'clf__features':[500], 'clf__mid_size':[100, 20]},
#             {'ufs__param':[1000], 'clf__features':[1000], 'clf__mid_size':[200, 100]},]

parameters = {'ufs__param':[200], 'clf__features':[200], 'clf__mid_size':[50]}
              
grid = find_best(pipeline, X, y, parameters, score, cv=5, verbose=1, n_jobs=-1)

pandas.DataFrame(grid.cv_results_)[['mean_fit_time','mean_score_time','params','mean_test_score','std_test_score']]

In [ ]:
results

## Deep Learning Model Grid search with CV

In [188]:
# Grid Search for parameters in deep learning models
# Careful here too. If you use oversample the cross validation might give better results then you would expect
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import numpy

score = make_scorer(balanced_accuracy_score)

# Function to create model, required for KerasClassifier
def create_model(optimizer='adam', init='glorot_uniform', dropout=0.2):
    num_features=1000
  
    # Model Definition
    model = Sequential()
    model.add(Dense(128, activation="relu", kernel_initializer=init, input_shape=(num_features,)))
#     model.add(Conv1D(filters=64, kernel_size=5, activation='relu', input_shape=(1000,)))
    model.add(Dropout(dropout))
    model.add(Dense(20, kernel_initializer=init, activation="relu"))
    model.add(Dense(3, kernel_initializer=init, activation="softmax"))

    # compile model using accuracy to measure model performance, tjese accuracies only show 
    # accuries of trained data which is too high...
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'], 
                  weighted_metrics=['acc'])

    model.summary()
    return model

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# create model
model = KerasClassifier(build_fn=create_model, verbose=2)

# Alternative to over/undersampling, we can also modify the weights in the loss function to give more importance
# to underrepresented classes in training. This is cheaper than oversampling and doesn't throw away data
# This only makes somewhat sense if you didn't under/oversample the data. Otherwise turn this off
if not undersample and not oversample:
    class_weight = {0: 6.,
                    1: 1.,
                    2: 6.}
else:
    class_weight = {0: 1.,
                    1: 1.,
                    2: 1.}
print("Class_weight", class_weight)
# grid search epochs, init, batch size and optimizer
optimizers = ['adam']#, 'rmsprop']
init = ['glorot_uniform']#, 'normal',]# 'uniform']
dropouts = [0.3]
epochs = [5, 15]#, 10]#, 100, 150]
batches = [100]#, 20]
param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=init, dropout=dropouts)

# set n_jobs to -1 to run on all cores but you lose out on verboseness for some reason
grid = GridSearchCV(estimator=model, param_grid=param_grid, verbose=2, scoring=score, n_jobs=1, cv=3)
grid_result = grid.fit(X, y, class_weight=class_weight)


# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))

Class_weight {0: 6.0, 1: 1.0, 2: 6.0}
Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] batch_size=100, dropout=0.3, epochs=5, init=glorot_uniform, optimizer=adam 
Model: "sequential_155"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_481 (Dense)            (None, 128)               128128    
_________________________________________________________________
dropout_246 (Dropout)        (None, 128)               0         
_________________________________________________________________
dense_482 (Dense)            (None, 20)                2580      
_________________________________________________________________
dense_483 (Dense)            (None, 3)                 63        
Total params: 130,771
Trainable params: 130,771
Non-trainable params: 0
_________________________________________________________________


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/5
 - 1s - loss: 2.0659 - accuracy: 0.5629 - acc: 0.5727
Epoch 2/5
 - 0s - loss: 1.5780 - accuracy: 0.6594 - acc: 0.6857
Epoch 3/5
 - 0s - loss: 1.3802 - accuracy: 0.7094 - acc: 0.7451
Epoch 4/5
 - 0s - loss: 1.1587 - accuracy: 0.7570 - acc: 0.7872
Epoch 5/5
 - 0s - loss: 1.0467 - accuracy: 0.7969 - acc: 0.8234
[CV]  batch_size=100, dropout=0.3, epochs=5, init=glorot_uniform, optimizer=adam, total=  21.2s
[CV] batch_size=100, dropout=0.3, epochs=5, init=glorot_uniform, optimizer=adam 
Model: "sequential_156"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_484 (Dense)            (None, 128)               128128    
_________________________________________________________________
dropout_247 (Dropout)        (None, 128)               0         
_________________________________________________________________
dense_485 (Dense)            (None, 20)                2580      
_________________

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   21.2s remaining:    0.0s


Epoch 1/5
 - 1s - loss: 2.1253 - accuracy: 0.5559 - acc: 0.5841
Epoch 2/5
 - 0s - loss: 1.5048 - accuracy: 0.6684 - acc: 0.7106
Epoch 3/5
 - 0s - loss: 1.3580 - accuracy: 0.7195 - acc: 0.7379
Epoch 4/5
 - 0s - loss: 1.1700 - accuracy: 0.7781 - acc: 0.7930
Epoch 5/5
 - 0s - loss: 1.0434 - accuracy: 0.7965 - acc: 0.8126
[CV]  batch_size=100, dropout=0.3, epochs=5, init=glorot_uniform, optimizer=adam, total=  21.4s
[CV] batch_size=100, dropout=0.3, epochs=5, init=glorot_uniform, optimizer=adam 
Model: "sequential_157"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_487 (Dense)            (None, 128)               128128    
_________________________________________________________________
dropout_248 (Dropout)        (None, 128)               0         
_________________________________________________________________
dense_488 (Dense)            (None, 20)                2580      
_________________

[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:  2.3min finished


Epoch 1/5
 - 1s - loss: 1.8731 - accuracy: 0.6383 - acc: 0.6141
Epoch 2/5
 - 0s - loss: 1.4580 - accuracy: 0.7182 - acc: 0.7193
Epoch 3/5
 - 0s - loss: 1.3193 - accuracy: 0.7427 - acc: 0.7486
Epoch 4/5
 - 0s - loss: 1.1601 - accuracy: 0.7812 - acc: 0.7905
Epoch 5/5
 - 0s - loss: 0.9878 - accuracy: 0.8159 - acc: 0.8365
Best: 0.680455 using {'batch_size': 100, 'dropout': 0.3, 'epochs': 5, 'init': 'glorot_uniform', 'optimizer': 'adam'}
0.680455 (0.006285) with: {'batch_size': 100, 'dropout': 0.3, 'epochs': 5, 'init': 'glorot_uniform', 'optimizer': 'adam'}
0.647297 (0.015339) with: {'batch_size': 100, 'dropout': 0.3, 'epochs': 15, 'init': 'glorot_uniform', 'optimizer': 'adam'}


In [ ]:
# Recreate model with best parameters and export prediction to CSV

def create_model(optimizer='adam', init='glorot_uniform', dropout=0.2):
    num_features=1000
  
    # Model Definition
    model = Sequential()
    regulizer = keras.regularizers.l2(0.000001)
    model.add(Dense(num_features, activation="relu", kernel_initializer=init, input_shape=(num_features,), kernel_regularizer=regulizer))
    # model.add(Conv1D(filters=64, kernel_size=5, activation='relu', input_shape=(1000,)))
    model.add(Dropout(dropout))
    model.add(Dense(50, kernel_initializer=init, activation="relu", kernel_regularizer=regulizer))
    model.add(Dropout(dropout))
    model.add(Dense(3, kernel_initializer=init, activation="softmax", kernel_regularizer=regulizer))

    #compile model using accuracy to measure model performance
    model.compile(optimizer=optimizer, loss='categorical_crossentropy')#, metrics=['accuracy'], weighted_metrics=['acc'])

    model.summary()
    return model

# Be careful to use the exact same settings
model = KerasClassifier(build_fn=create_model)
model.fit(X,Y, epochs=5, batch_size=10)


In [ ]:
# Export to CSV
y_test = model.predict(Xtest)
df = pd.DataFrame(y_test, columns=["y"])
df.index.name = "id"
df.to_csv("results_neural_net_try2.csv")

## Deep learning with cross validation using train_test_split and on the fly cross validation. 
For this one use test_train split in the beginning

In [ ]:
import keras

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, GaussianNoise
from keras.optimizers import SGD
import keras.backend as K
from sklearn.metrics import roc_auc_score


# This is tricky when you oversample, because the training data will still look similar to the validation data
# x_train, x_val, y_train, y_val = train_test_split(
#     X, y, test_size=0.5)
# Therefore I moved the splitting part to before the oversampling


class Metrics(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self._data = []

    def on_epoch_end(self, batch, logs={}):
        X_val, y_val = self.validation_data[0], self.validation_data[1]
        y_predict = np.asarray(model.predict(X_val))

        y_val = np.argmax(y_val, axis=1)
        y_predict = np.argmax(y_predict, axis=1)

        self._data.append(
            balanced_accuracy_score(y_val, y_predict),
        )
        return

    def get_data(self):
        return self._data
    
y_trains = keras.utils.to_categorical(y, num_classes=3)
y_tests = keras.utils.to_categorical(y_val, num_classes=3)


model = Sequential()
act = keras.layers.LeakyReLU(alpha=0.3)
#act = keras.layers.ThresholdedReLU(theta=1.0)
#regulizer = keras.regularizers.l1_l2(l1=0.01, l2=0.01)
# regulizer = keras.regularizers.l1(0.00001)
regulizer = keras.regularizers.l2(0.000001)
model.add(Dense(128, activation="relu",kernel_initializer='he_normal', kernel_regularizer=regulizer,input_dim=1000))
model.add(Dropout(0.3))
model.add(GaussianNoise(12))
model.add(Dense(32, kernel_initializer='he_normal',activation=act,kernel_regularizer=regulizer))
model.add(GaussianNoise(15))

model.add(Dense(16, kernel_initializer='he_normal',activation=act,kernel_regularizer=regulizer))
model.add(Dropout(0.3))
model.add(Dense(3, activation='softmax',kernel_initializer='he_normal'))


sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
rmsprop = keras.optimizers.RMSprop(lr=0.001, rho=0.9, decay=0.0)
model.compile(loss='categorical_crossentropy',
              optimizer=rmsprop,
              metrics=['acc'])


metrics = Metrics()
history = model.fit(X, y_trains, validation_data=(x_val, y_tests), epochs=30, batch_size=200, verbose=2,callbacks=[metrics])
metrics.get_data()
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(metrics.get_data())
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()



/home/sed/anaconda2/envs/lasif_2.0/lib/python3.6/site-packages/keras/activations.py:235: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


Train on 3840 samples, validate on 960 samples
Epoch 1/30
 - 3s - loss: 21.3924 - acc: 0.4281 - val_loss: 0.7402 - val_acc: 0.7552
Epoch 2/30
 - 0s - loss: 12.7832 - acc: 0.5771 - val_loss: 1.0393 - val_acc: 0.7573
Epoch 3/30
 - 0s - loss: 9.2766 - acc: 0.6148 - val_loss: 1.0953 - val_acc: 0.7667
Epoch 4/30
 - 0s - loss: 7.5114 - acc: 0.6276 - val_loss: 1.1029 - val_acc: 0.7833
Epoch 5/30
 - 0s - loss: 6.7314 - acc: 0.6365 - val_loss: 1.0685 - val_acc: 0.7781
Epoch 6/30
 - 0s - loss: 5.3106 - acc: 0.6542 - val_loss: 0.9885 - val_acc: 0.7896
Epoch 7/30
 - 0s - loss: 4.8892 - acc: 0.6583 - val_loss: 0.9269 - val_acc: 0.7365
Epoch 8/30
 - 0s - loss: 4.3798 - acc: 0.6529 - val_loss: 0.8786 - val_acc: 0.7229
Epoch 9/30
 - 0s - loss: 3.4970 - acc: 0.6693 - val_loss: 0.8561 - val_acc: 0.6958
Epoch 10/30
 - 0s - loss: 3.2414 - acc: 0.6695 - val_loss: 0.8959 - val_acc: 0.6885
Epoch 11/30
 - 0s - loss: 2.8799 - acc: 0.6727 - val_loss: 0.7943 - val_acc: 0.7042
Epoch 12/30
 - 0s - loss: 2.5422 - a

In [166]:
# print final cross validation score
y_pred = model.predict(x_val)
balanced_accuracy_score(y_val, y_pred.argmax(axis=1))

0.6860968666720503

In [171]:
# After finding proper settings retrain on the entire dataset
# make predictions and rwrite to CSV
y_all_cats = keras.utils.to_categorical(y_all, num_classes=3)
model.fit(X_all, y_all, validation_data=(x_val, y_tests), epochs=25, batch_size=200, verbose=2,callbacks=[metrics])

ypred = model.predict(Xtest)
y_test = ypred.argmax(axis=1)

# Export to CSV
if len(y_test) == 4100: 
    df = pd.DataFrame(y_test, columns=["y"])
    df.index.name = "id"
    df.to_csv("results_neural_net_new_cv2.csv")
else: 
    raise Exception("check length")

Train on 4800 samples, validate on 960 samples
Epoch 1/25
 - 0s - loss: 0.4796 - acc: 0.7900 - val_loss: 0.4904 - val_acc: 0.7865
Epoch 2/25
 - 0s - loss: 0.4603 - acc: 0.7994 - val_loss: 0.4781 - val_acc: 0.7948
Epoch 3/25
 - 0s - loss: 0.4566 - acc: 0.7996 - val_loss: 0.4781 - val_acc: 0.7927
Epoch 4/25
 - 0s - loss: 0.4544 - acc: 0.8140 - val_loss: 0.4607 - val_acc: 0.8188
Epoch 5/25
 - 0s - loss: 0.4420 - acc: 0.8173 - val_loss: 0.4477 - val_acc: 0.8260
Epoch 6/25
 - 0s - loss: 0.4444 - acc: 0.8183 - val_loss: 0.4509 - val_acc: 0.8167
Epoch 7/25
 - 0s - loss: 0.4369 - acc: 0.8190 - val_loss: 0.4435 - val_acc: 0.8229
Epoch 8/25
 - 0s - loss: 0.4279 - acc: 0.8208 - val_loss: 0.4379 - val_acc: 0.8198
Epoch 9/25
 - 0s - loss: 0.4303 - acc: 0.8215 - val_loss: 0.4382 - val_acc: 0.8177
Epoch 10/25
 - 0s - loss: 0.4237 - acc: 0.8252 - val_loss: 0.4278 - val_acc: 0.8271
Epoch 11/25
 - 0s - loss: 0.4161 - acc: 0.8173 - val_loss: 0.4243 - val_acc: 0.8302
Epoch 12/25
 - 0s - loss: 0.4081 - acc

In [170]:
np.shape(y_all)

(4800, 3)

In [182]:
not undersample

True

In [185]:
oversample

False

In [189]:
grid_result.cv_results_

{'mean_fit_time': array([17.60451023, 20.15859119]),
 'std_fit_time': array([0.23480723, 0.23574864]),
 'mean_score_time': array([3.92661866, 4.10377733]),
 'std_score_time': array([0.18482539, 0.18425059]),
 'param_batch_size': masked_array(data=[100, 100],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'param_dropout': masked_array(data=[0.3, 0.3],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'param_epochs': masked_array(data=[5, 15],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'param_init': masked_array(data=['glorot_uniform', 'glorot_uniform'],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'param_optimizer': masked_array(data=['adam', 'adam'],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'params': [{'batch_size': 100,
   'dropout': 0.3,
   'epochs': 5,
   'init': 'glorot_u